## Chain of Responsibility


**Used when you want to pass the same `Object` to various `Handlers`**
- Each handler should take `Object` as input and return it back or call another handler
- Each Handler can do some work or skip also


**Note**:
- In the below example, we also let the `Handler` store the next handler and call it ( so that we can chain it )
- But it can also work like, `object` -> `Handler` -> `object` ( here you need to manually do the next steps )


**How is different from Builder Pattern?**
- In builder we use chaining but to create object


In [ ]:
from abc import ABC, abstractmethod


class Handler(ABC):
    def __init__(self):
        self._next = None

    def set_next(self, handler):
        self._next = handler
        return handler  # allows chaining: a.set_next(b).set_next(c)

    @abstractmethod
    def handle(self, request: dict):
        pass

    def next_handler(self, request: dict): # CALL THE NEXT HANDLER
        if self._next:
            return self._next.handle(request)
        return "Request passed all checks."


In [4]:
class AuthHandler(Handler):
    def handle(self, request):
        if not request.get("user"):
            return "Authentication failed: No user."
        print("Auth OK")
        return self.next_handler(request)


class PermissionHandler(Handler):
    def handle(self, request):
        if not request.get("role") == "admin":
            return "Permission denied: Admin role required."
        print("Permission OK")
        return self.next_handler(request)


class DataValidationHandler(Handler):
    def handle(self, request):
        if request.get("payload") is None:
            print("SKIP DATA VALIDATION: Payload is None")
        elif "payload" not in request or not request["payload"]:
            return "Invalid request: Missing payload."
        print("Data OK")
        return self.next_handler(request)


In [ ]:
# Build the chain: Auth → Permission → Data Validation
auth = AuthHandler()
auth.set_next(PermissionHandler()).set_next(DataValidationHandler())

request = {
    "user": "alex",
    "role": "admin",
    "payload": {"item": "Laptop"}
}

result = auth.handle(request)
print(result,"\n")

request = {
    "user": "alex",
    "role": "admin",
    "payload": None
}

result = auth.handle(request)
print(result)


Auth OK
Permission OK
Data OK
Request passed all checks. 

Auth OK
Permission OK
SKIP DATA VALIDATION: Payload is None
Data OK
Request passed all checks.


## Command

**Intuition**
- Whenever you have a system where you can give multiple commands to it -> use this pattern
- IDEA: Your request itself is an object of `Command` -> the command/request should have all the data needed to do the command

**Example 1: Splitwise**
- In splitwise you have the following possible commands/operations - addUser, removeUser, addExpense
- I treat each of this as a command -> the command can then call some specific method to handle it

**Example 2: TextEditor**
- You build a GUI Text editor, this has multiple buttons which do different things
- Consider you have a backend that implements various operations as API endpoints
- Solution 1:  
  1. Have a base `Button` class and let all the other buttons extend this + implement their own logic
  2. Cons: You can lots of buttons and each define their logic by themselves ( too many to handle, some may have common functionalities)
- Solution:
  - Lets not allow each button to define how to call
  - Let each button emit a particular `Command` and then let it handle
  



**Implementation**
1. `command` -> the request/command
2. `Invoker` -> the class that emits/gives this commands
   1. splitwise: UIEventHandler
   2. textEditor: Each button in the UI is an invoker ( can invoke different commands )
3. `receiver` -> class which recieves the command and acts upon it

In [9]:
from abc import ABC, abstractmethod

class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

    @abstractmethod
    def undo(self): 
        pass


In [11]:
# RECEIVER
class SplitwiseSystem:
    def add_user(self, user): print(f"[ADD USER] {user}")
    def remove_user(self, user): print(f"[REMOVE USER] {user}")
    def add_expense(self, payer, amount): print(f"[EXPENSE] {payer} paid {amount}")


In [12]:
class AddUserCommand(Command):
    def __init__(self, system, user):
        self.system = system
        self.user = user

    def execute(self): self.system.add_user(self.user)
    def undo(self): self.system.remove_user(self.user)


class RemoveUserCommand(Command):
    def __init__(self, system, user):
        self.system = system
        self.user = user

    def execute(self): self.system.remove_user(self.user)
    def undo(self): self.system.add_user(self.user)


class AddExpenseCommand(Command):
    def __init__(self, system, payer, amount):
        self.system = system
        self.payer = payer
        self.amount = amount

    def execute(self): self.system.add_expense(self.payer, self.amount)
    def undo(self): print(f"[UNDO EXPENSE] Refund {self.payer}: {self.amount}")


In [13]:
# INVOKER
class CommandInvoker:
    def __init__(self):
        self.history = []

    def run(self, command):
        command.execute()
        self.history.append(command)

    def undo(self):
        if not self.history:
            print("Nothing to undo.")
            return
        last = self.history.pop()
        last.undo()


In [14]:
system = SplitwiseSystem()
invoker = CommandInvoker()

invoker.run(AddUserCommand(system, "Alice"))
invoker.run(AddUserCommand(system, "Bob"))
invoker.run(AddExpenseCommand(system, "Alice", 50))

print("---- UNDO ACTION ----")
invoker.undo()  # Undo expense
invoker.undo()  # Undo add Bob
invoker.undo()  # Undo add Alice


[ADD USER] Alice
[ADD USER] Bob
[EXPENSE] Alice paid 50
---- UNDO ACTION ----
[UNDO EXPENSE] Refund Alice: 50
[REMOVE USER] Bob
[REMOVE USER] Alice


## Iterator

## Mediator

## Memento

## Observer

1. `OBSERVER` 
    - This is the consumer/container. You subscribe to a `Observable`
    - This has a `notify` method that the `Observable` object calls in case of any changes. It can either be called only with the `change_event` but can also be called with the `Observable` object itself if more info is needed
    - **You can either subscribe manually or pass list of Observables to subscribe in the init function**

2. `OBSERVABLE`
   - Stores the list/set of observers subscribed to it
   - `subscribe()`
   - `unsubscribe()`
   - `notify()` -> in case of change update all of its `Observers`

In [ ]:
# when you want to refer future Classes in type hint -> put them in strings
from abc import ABC

class Observer(ABC):
    def notify(observable: "Observable", change_event: str) -> None:
        # The observer was notified about event  from Observable
        pass

class Observable(ABC):
    @staticmethod
    def subscribe(observer: Observer) -> None:
        pass

    @staticmethod
    def unsubscribe(observer: Observer) -> None:
        pass

    @staticmethod
    def change_event(self) -> None: 
        # Notify all observers about the change event -> pass event description + itself
        pass 

In [19]:
class Stock(Observable):
    def __init__(self, name: str, price: float) -> None:
        self.name = name
        self.price = price
        self.observers = set()

    def subscribe(self, observer: Observer) -> None:
        self.observers.add(observer)
    
    def unsubscribe(self, observer: Observer) -> None:
        self.observers.remove(observer)

    def change_event(self, new_price: float) -> None:
        self.price = new_price
        for observer in self.observers:
            observer.notify(self, f"Price changed to {new_price}")


class WebAppObserver(Observer):
    def notify(self, observable: Observable, change_event: str) -> None:
        print(f"{observable.name}: Notified about change {change_event} in Web App")

class MobileAppObserver(Observer):
    def notify(self, observable: Observable, change_event: str) -> None:
        print(f"{observable.name}: Notified about change {change_event} in Mobile App")

In [20]:
apple_stock = Stock("AAPL", 150.0)
msft_stock = Stock("MSFT", 250.0)

web_observer = WebAppObserver()
mobile_observer = MobileAppObserver()

# SUBSCRIPTION
apple_stock.subscribe(web_observer)
apple_stock.subscribe(mobile_observer)

msft_stock.subscribe(mobile_observer)

# PRICE CHANGE EVENT
apple_stock.change_event("Price increased to 155.0")
msft_stock.change_event("Price decreased to 245.0")


AAPL: Notified about change Price changed to Price increased to 155.0 in Web App
AAPL: Notified about change Price changed to Price increased to 155.0 in Mobile App
MSFT: Notified about change Price changed to Price decreased to 245.0 in Mobile App


In [21]:
# IMPROVEMENTS: You can pass list of observables to observer during initialization -> NO EXTRA STEP TO SUBSCRIBE

class WebAppObserverBetter(Observer):
    def __init__(self, observables: list[Observable]) -> None:
        self.observables = observables
        for observable in observables:
            observable.subscribe(self)

    def notify(self, observable: Observable, change_event: str) -> None:
        print(f"{observable.name}: Notified about change {change_event} in Web App")

class MobileAppObserverBetter(Observer):
    def __init__(self, observables: list[Observable]) -> None:
        self.observables = observables
        for observable in observables:
            observable.subscribe(self)

    def notify(self, observable: Observable, change_event: str) -> None:
        print(f"{observable.name}: Notified about change {change_event} in Mobile App")

In [22]:
apple_stock = Stock("AAPL", 150.0)
msft_stock = Stock("MSFT", 250.0)

web_observer = MobileAppObserverBetter([apple_stock, msft_stock])
mobile_observer = MobileAppObserverBetter([apple_stock])


# PRICE CHANGE EVENT
apple_stock.change_event("Price increased to 155.0")
msft_stock.change_event("Price decreased to 245.0")


AAPL: Notified about change Price changed to Price increased to 155.0 in Mobile App
AAPL: Notified about change Price changed to Price increased to 155.0 in Mobile App
MSFT: Notified about change Price changed to Price decreased to 245.0 in Mobile App


## State

## Strategy

## Template Method

## Visitor